In [15]:
import pandas as pd
import networkx as nx
from pathlib import Path

In [16]:
WORKBOOK_PATH = Path('../../data/assemblies.xlsx')
OUTPUT_PATH   = WORKBOOK_PATH

In [17]:
df = pd.read_excel(WORKBOOK_PATH, sheet_name=0)
df.columns = df.columns.str.strip()                # remove any leading/trailing spaces
required_cols = {'ID', 'PredecessorIDs', 'SuccessorIDs'}           
if not required_cols.issubset(df.columns):
    raise KeyError(f"Missing one of {required_cols} in {df.columns.tolist()}")

In [18]:
# helper to split comma-lists
def parse_ids(cell):
    s = str(cell).strip()
    if not s or s.lower() == 'nan':
        return []
    return [int(x.strip()) for x in s.split(',') if x.strip()]

In [19]:
# — 2) collect edges from predecessors AND successors —
edge_set = set()
for _, row in df.iterrows():
    tid = int(row['ID'])

    # predecessor → this task
    for p in parse_ids(row['PredecessorIDs']):
        edge_set.add((p, tid))

    # this task → successor
    if 'SuccessorIDs' in df.columns:
        for s in parse_ids(row['SuccessorIDs']):
            edge_set.add((tid, s))

edges = list(edge_set)

print(f"Collected {len(edges):,} edges")
if edges:
    print("First 10 edges:", edges[:10])
else:
    print("⚠️  No edges were collected – check your column names or cell values!")

Collected 540 edges
First 10 edges: [(96, 95), (177, 181), (110, 2), (55, 75), (188, 187), (207, 200), (118, 122), (59, 82), (129, 137), (72, 2)]


In [20]:
# — 3) build graph & detect cycles —
G = nx.DiGraph()
G.add_edges_from(edges)

cycles = list(nx.simple_cycles(G))
print(f'Found {len(cycles)} cycles')

Found 0 cycles


In [21]:
# — build human‐readable cycle strings (ID-based) if you haven’t already —
cycle_strings = [' → '.join(map(str, c + [c[0]])) for c in cycles]

# — map IDs to descriptions —
id_to_desc = df.set_index('ID')['Description'].astype(str).to_dict()

# — build named‐cycle strings —
named_strings = [
    " → ".join(id_to_desc.get(n, f"ID {n}") for n in c + [c[0]])
    for c in cycles
]

# — assemble DataFrame —
named_df = pd.DataFrame({
    'Cycle':       cycle_strings,
    'Cycle_Named': named_strings
})

# — write to Excel sheet “Cycles_Named” —
with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    named_df.to_excel(writer, sheet_name='Cycles_Named', index=False)

print(f"Wrote {len(named_df)} cycles → sheet 'Cycles_Named'")

Wrote 0 cycles → sheet 'Cycles_Named'
